In [1]:
from mlaut.analyze_results import AnalyseResults
from mlaut.data import Data
from mlaut.data.data import Data
import pickle
import numpy as np
from sklearn.metrics import mean_squared_error
from mlaut.experiments import Orchestrator
from mlaut.estimators.estimators import instantiate_default_estimators
import os

/media/viktor/Data/PhD/mlaut/py36/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = Data()
input_io = data.open_hdf5('data/openml.h5', mode='r')
out_io = data.open_hdf5('data/openml-classification.h5', mode='a')

In [3]:
dts_names = os.listdir('data/trained_models')
orchest = Orchestrator(hdf5_input_io=input_io, 
                           hdf5_output_io=out_io,
                           dts_names=dts_names,
                           original_datasets_group_h5_path='/openml')

In [4]:
instantiated_models = instantiate_default_estimators(estimators=['NeuralNetworkDeepClassifier'], verbose=1)
orchest.predict_all(trained_models_dir='data/trained_models', estimators=instantiated_models)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


2018-05-16 22:33:22,696 [MainThread  ] [WARNI]  From /media/viktor/Data/PhD/mlaut/py36/local/lib/python3.6/site-packages/tensorflow/python/keras/_impl/keras/backend.py:1557: calling reduce_mean (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
2018-05-16 22:33:22,696 [MainThread  ] [WARNI]  From /media/viktor/Data/PhD/mlaut/py36/local/lib/python3.6/site-packages/tensorflow/python/keras/_impl/keras/backend.py:1557: calling reduce_mean (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
2018-05-16 22:33:22,696 [MainThread  ] [WARNI]  From /media/viktor/Data/PhD/mlaut/py36/local/lib/python3.6/site-packages/tensorflow/python/keras/_impl/keras/backend.py:1557: calling reduce_mean (from tensorflow.python.ops.math_ops) with keep_dims is 

***********[1 1 0 0 1 1 1 0 1 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 1 1 1 0 0 0 1 0 0 0
 0 0 0 1 0 1 1 0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 0 0 1 0 0 0 1 0 0 1 1 1 0 1 0
 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 0 0 0 1 1 0 1 0
 1 1 0 1 1 0 0 0 0 1 1 1 0 1 0 1 1 1 0 1 1 0 0 0 1 0 0 1 0 0 0 1 0 1 1 0 0
 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 1 0 0 0 1 1 0 1 0 0 1 0 1
 0 1 0 1 1 0 0 0 0 1 1 1 0 1 1 1 1 1 0 0 0 0 0]


ValueError: Save path already exists

### Debugging

In [ ]:
from tensorflow.python.keras import models
from tensorflow.python.keras.models import Sequential
import os
from sklearn.preprocessing import OneHotEncoder
from mlaut.data import Data


In [ ]:
data = Data()
input_io = data.open_hdf5('data/openml.h5', mode='r')
out_io = data.open_hdf5('data/openml-classification.h5', mode='a')

In [ ]:
class OverwrittenSequentialClassifier(Sequential):
    """
    Keras sequential model that overrides the default :func:`tensorflow.python.keras.models.fit` and :func:`tensorflow.python.keras.models.predict` methods.
    """



    def fit(self, X_train, y_train):
            
        """
        Overrides the default :func:`tensorflow.python.keras.models.fit` and reshapes the `y_train` in one hot array. 

        Paremeters
        ----------
        X_train: training data
        y_train: Labels that will be converted to onehot array.


        Returns
        -------
        :func:`tensorflow.python.keras.models.fit` object

        """
        onehot_encoder = OneHotEncoder(sparse=False)
        len_y = len(y_train)
        reshaped_y = y_train.reshape(len_y, 1)
        y_train_onehot_encoded = onehot_encoder.fit_transform(reshaped_y)
        
        
        return super().fit(X_train, y_train_onehot_encoded)

    def pred(self, X_test):
        """
        Overrides the default :func:`tensorflow.python.keras.models.predict` by replacing it with a :func:`tensorflow.python.keras.models.predict_classes`  

        Returns
        --------
        :func:`tensorflow.python.keras.models.predict_classes`
        """

        return Sequential.predict_classes(X_test)
        # return super().predict_classes(X_test)

In [ ]:
mn = models.load_model('data/trained_models/AP_Breast_Colon/NeuralNetworkDeepClassifier.h5',
                  custom_objects={
                               'OverwrittenSequentialClassifier':OverwrittenSequentialClassifier
                               })

In [ ]:
datasets = os.listdir('data/trained_models/')

In [ ]:
(X_train, 
 X_test, 
 y_train, 
 y_test) = data.load_test_train_dts(hdf5_out=out_io,
                                    hdf5_in=input_io, 
                                    dts_name=datasets[0], 
                                    dts_grp_path='/openml')

In [ ]:
mn.pred(X_test)

In [ ]:
import numpy as np
a = np.array([[1,0],[1,0],[0,1]])

In [ ]:
a.argmax(axis=1)